In [17]:
!pip install numpy
!pip install gym
!pip install matplotlib
!pip install sklearn
!pip install pyglet
import numpy as np
import time
import math
import random
import gym
from typing import Tuple
import matplotlib.pyplot as plt
from IPython import display
from sklearn.preprocessing import KBinsDiscretizer

  Using cached pyglet-1.5.21-py3-none-any.whl (1.1 MB)


## CartPole-V1 Enviornment Setup

In [6]:
env = gym.make('CartPole-v1')

## Enviornment Documentation

In [3]:
?env.env

## Visualization

In [4]:
#Hard coded policy value
policy = lambda obs: 0
for i in range(5):
    obs = env.reset()
    for j in range(30):
        actions = policy(obs)
        obs, reward, done, info = env.step(actions)
        env.render()
        ## IF WANT TO PRESENT ANIMATION IN JUPYTER
        #plt.imshow(env.render(mode='rgb_array'))
        #display.display(plt.gcf())    
        #display.clear_output(wait=True)
        time.sleep(0.1)
env.close()

/home/saeedbai/.local/lib/python3.8/site-packages/gym/logger.py:34: UserWarning: WARN: You are calling 'step()' even though this environment has already returned done = True. You should always call 'reset()' once you receive 'done = True' -- any further steps are undefined behavior.
  warnings.warn(colorize("%s: %s" % ("WARN", msg % args), "yellow"))


KeyboardInterrupt: 

## Policy Function

In [52]:
#policy = lambda _,__,___, tip_velocity : int( tip_velocity > 0 )

## Buckets and Actions

In [64]:
#n_bucket = (1,1,6,3)
n_bucket = (6,12)
n_actions = env.action_space.n

## Reseting boundarys

In [65]:
state_val_bounds = list(zip(env.observation_space.low,env.observation_space.high))
#Reset cart velocity
state_val_bounds[1] = [-0.5,0.5]
#Reset pole angular velocity
state_val_bounds[3] = [-math.radians(50),math.radians(50)]
print(state_val_bounds[2][0])

-0.41887903


## Action index and Q value

In [66]:
act_index = len(n_bucket)
q_val_table = np.zeros(n_bucket+(n_actions,))
q_val_table.shape


(6, 12, 2)

## Define rate

In [67]:
min_exp_rate = 0.1
min_learn_rate = 0.01

## INITIALIZATIONS

In [68]:
episodes = 2000
discount = 1

In [69]:
## This function choose what action to perform
def sel_act(sel_val,exp_rate):
    #epsilon greedy algorithm
    #exploration and exploitation
    if random.random() < exp_rate:
        action = env.action_space.sample()
    else:
        action = np.argmax(q_val_table[sel_val])
    return action

In [70]:
## This function choose what explore rate to use
def sel_exp_rate(input):
    return max(min_exp_rate,min(1,1.0 - math.log10((input+1)/25)))
## This function choose what learn rate to use
def sel_learn_rate(input):
    return max(min_learn_rate,min(1.0,1.0 - math.log10((input+1)/25)))

In [71]:
## Discrete state value
def discrete_state_val(stateval):
    lower_bounds = [ env.observation_space.low[2], -math.radians(50) ]
    upper_bounds = [ env.observation_space.high[2], math.radians(50) ]
    est = KBinsDiscretizer(n_bins=n_bucket, encode='ordinal', strategy='uniform')
    est.fit([lower_bounds, upper_bounds ])
    return tuple(map(int,est.transform([[stateval[2], stateval[3]]])[0]))

In [72]:
## Update Q value
def update_q_val(best_q_val,reward):
    # Temporal difference
    return discount * best_q_val + reward
    

## Training

In [ ]:
for episode in range(episodes):
    # counter for rewards earned in each episode
    episode_reward = 0
    
    learn_rate = sel_learn_rate(episode)
    exp_rate = sel_exp_rate(episode)
    # reset enviornment
    obs = env.reset()
    init_state_val = discrete_state_val(obs)
    done = False
    
    # each episode
    while done == False:
        
        action = sel_act(init_state_val,exp_rate)

        obs, reward, done, info = env.step(action)
        
        state_val = discrete_state_val(obs)
        
        best_q_val = np.max(q_val_table[state_val])
        learnt_val = update_q_val(best_q_val,reward)
        
        old_val = q_val_table[init_state_val][action]
        
        # Bellman equation
        q_val_table[init_state_val][action] = (1 - learn_rate) * old_val + learnt_val * learn_rate

        init_state_val = state_val
        env.render()
    
        episode_reward += reward
        
    print(episode_reward)
env.close()

TypeError: render() got an unexpected keyword argument 'close'